In [1]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [36]:
train_set = pd.read_csv('train/train_encoded_sin_location.csv')
train_set.head(10)

,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8,id,len_text,len_text_original,diff_len_text,text_sum,text_min,text_max,text_mean,text_median,target
0,0,0,0,0,0,0,0,1,1,43,69,26,2.745943,0.224894,0.474996,0.343243,0.323476,1
1,0,0,0,0,0,0,0,1,4,36,38,2,2.545323,0.211650,0.508967,0.363618,0.340166,1
2,0,0,0,0,0,0,0,1,5,72,133,61,3.925010,0.153758,0.585124,0.327084,0.256037,1
3,0,0,0,0,0,0,0,1,6,50,65,15,2.515355,0.150085,0.574226,0.359336,0.346962,1
4,0,0,0,0,0,0,0,1,7,52,88,36,2.937988,0.247769,0.444024,0.326443,0.287533,1
5,0,0,0,0,0,0,0,1,8,84,110,26,3.472170,0.094812,0.392155,0.267090,0.268381,1
6,0,0,0,0,0,0,0,1,10,74,95,21,3.652263,0.211903,0.421501,0.304355,0.292368,1
7,0,0,0,0,0,0,0,1,13,21,59,38,2.169400,0.288043,0.580492,0.433880,0.410864,1
8,0,0,0,0,0,0,0,1,14,38,79,41,2.425554,0.344773,0.491582,0.404259,0.389401,1
9,0,0,0,0,0,0,0,1,15,24,52,28,1.966376,0.391599,0.639305,0.491594,0.467736,1


### XGBoost

In [37]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Random Search

In [31]:
learning_rate = [0.1, 0.3, 0.01, 0.001]
max_depth = [20, 50, 70, 100]
#subsamples = [0.4, 0.5, 0.6]
colsample_bytree = [0.4, 0.5, 0.6]
n_estimators = [20, 50, 70, 100]
objective = ['reg:logistic']

random_search = {'learning_rate': learning_rate, 'max_depth': max_depth,\
                'colsample_bytree': colsample_bytree, 'n_estimators': n_estimators, 'objective': objective}

In [32]:
xgb_random = RandomizedSearchCV(estimator = xgb.XGBClassifier(),\
                               param_distributions=random_search, n_iter=50, cv=7, random_state=111)

In [33]:
xgb_random.fit(X_train, y_train)

RandomizedSearchCV(cv=7,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                    

In [34]:
xgb_random.best_params_

{'objective': 'reg:logistic',
 'n_estimators': 100,
 'max_depth': 50,
 'learning_rate': 0.01,
 'colsample_bytree': 0.4}

In [35]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.5,\
                           learning_rate = 0.01, max_depth = 100, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.817390


##### Best params de random forest

In [7]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3, colsample_bytree = 0.6,\
                           learning_rate = 0.1, max_depth = 70, alpha = 10, n_estimators = 100)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.809586


##### Parámetros que mejor funcionaron con pruebas a mano

In [6]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 70, alpha = 10, n_estimators = 57)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.801706


## K-fold cross validation

In [25]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

params = {'objective': 'reg:logistic', 'colsample_bytree':0.3, 'learning_rate': 0.1, 'max_depth': 100,\
         'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10, num_boost_round=300, early_stopping_rounds=20,\
                   metrics=['error', 'rmse'], as_pandas=True, seed=42)

In [26]:
cv_results.head(10)

,train-error-mean,train-error-std,train-rmse-mean,train-rmse-std,test-error-mean,test-error-std,test-rmse-mean,test-rmse-std
0,0.328765,0.009732,0.493512,0.000347,0.368315,0.012067,0.494754,0.000463
1,0.300642,0.018129,0.488429,0.001778,0.349262,0.027324,0.490606,0.001844
2,0.295562,0.019460,0.485316,0.002212,0.348216,0.022955,0.488323,0.001674
3,0.285418,0.013733,0.481155,0.002328,0.342181,0.021163,0.485187,0.001586
4,0.278223,0.018346,0.477658,0.002607,0.336666,0.022073,0.482516,0.002015
5,0.272297,0.016595,0.473710,0.003332,0.328258,0.019915,0.479540,0.002234
6,0.267131,0.012999,0.470872,0.003126,0.326155,0.014699,0.477449,0.002143
7,0.262854,0.014262,0.467874,0.003140,0.324447,0.015538,0.475433,0.002158
8,0.261161,0.013853,0.465550,0.003424,0.324975,0.018504,0.473984,0.002131
9,0.257819,0.014634,0.463070,0.003560,0.319585,0.020750,0.472469,0.002119


In [27]:
cv_results['train-error-mean'].tail(1)

299    0.14233
Name: train-error-mean, dtype: float64

### Test

In [21]:
test_set = pd.read_csv('test/test_encoded_sin_location.csv')

In [22]:
preds = xg_reg.predict(test_set)
preds

array([0, 1, 1, ..., 0, 1, 1])

In [23]:
test_set['target'] = preds
test_set[['id', 'target']]

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,0
3261,10874,1


### Resultado

In [24]:
test_set[['id', 'target']].to_csv('result_xgb_sin_location.csv', header=True, index=False)